### Создание искусственных лиц при помощи генеративно-состязательных моделей
<img src="https://www.strangerdimensions.com/wp-content/uploads/2013/11/reception-robot.jpg" width=320>
Мы обучим нейронную сеть генерации правдоподобно выглядящих человеческих лиц во всём их разнообразии: разный внешний вид, выражения лица, аксессуары и так далее. После захвата Земли машинами, лиц больше не останется, и мы хотим сохранить эти данные дла будущих итераций. Жуть...

Основано на https://github.com/Lasagne/Recipes/pull/94

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
plt.rcParams.update({'axes.titlesize': 'small'})

from sklearn.datasets import load_digits
#The following line fetches you two datasets: images, usable for autoencoder training and attributes.
#Those attributes will be required for the final part of the assignment (applying smiles), so please keep them in mind
from lfw_dataset import fetch_lfw_dataset
data,attrs = fetch_lfw_dataset(dimx=36,dimy=36)

#preprocess faces
data = np.float32(data).transpose([0,3,1,2]) / 255.

IMG_SHAPE = data.shape[1:]

In [ ]:
#print random image
plt.imshow(data[np.random.randint(data.shape[0])].transpose([1,2,0]),
           cmap="gray", interpolation="none")

# Основы генеративно-состязательных сетей

<img src="https://raw.githubusercontent.com/torch/torch.github.io/master/blog/_posts/images/model.png" width=320px height=240px>

Глубинное обучение довольно простое, не так ли? 
* Создаём сеть, генерирующую лицо (изображение небольшого размера)
* Придумываем __метрику__ того, __насколько это хорошее лицо__
* Оптимизируем градиентным спуском :)


Единственная проблема заключается в определении того, как отличить хорошо сгенерированное лицо от плохого? В данном случае будет сложно найти специалиста для помощи. 

__Если мы не можем отличить хорошее лицо от плохого, мы делегируем задачу другой нейронной сети!__

Теперь у нас есть две сети:
* __G__ (генератор) - принимает на вход случайный шум и пытается сгенерировать пример лица.
  * Обозначим его как __G__(z), где z это гауссовский шум.
* __D__ (дискриминатор) - принимает на вход пример лица и определяет, является ли оно хорошим или сгенерированным.
  * Определяет вероятность того, что ему на вход было подано __настоящее лицо__
  * Обозначим его как __D__(x), где x это изображение.
  * __D(x)__ это предсказание для настоящего лица и __D(G(z))__ это предсказание для лица, созданного генератором.

Прежде чем мы приступим к обучению, давайте создадим обе сети.

In [ ]:
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

use_cuda = torch.cuda.is_available()

print("Torch version:", torch.__version__)
if use_cuda:
    print("Using GPU")
else:
    print("Not using GPU")

In [ ]:
def sample_noise_batch(batch_size):
    noise = Variable(torch.randn(batch_size, CODE_SIZE))
    return noise.cuda() if use_cuda else noise.cpu()
    
class Reshape(nn.Module):
    def __init__(self, shape):
        nn.Module.__init__(self)
        self.shape=shape
    def forward(self,input):
        return input.view(self.shape)

In [ ]:
CODE_SIZE = 256

from itertools import count
# automatic layer name maker. Don't do this in production :)
ix = ('layer_%i'%i for i in count())

generator = nn.Sequential()

generator.add_module(next(ix), nn.Linear(CODE_SIZE, 10*8*8))
generator.add_module(next(ix), nn.ELU())
generator.add_module(next(ix), Reshape([-1, 10, 8, 8]))

generator.add_module(next(ix), nn.ConvTranspose2d(10, 64, kernel_size=(5,5)))
generator.add_module(next(ix), nn.ELU())
generator.add_module(next(ix), nn.ConvTranspose2d(64, 64, kernel_size=(5,5)))
generator.add_module(next(ix), nn.ELU())
generator.add_module(next(ix), nn.Upsample(scale_factor=2))

generator.add_module(next(ix), nn.ConvTranspose2d(64, 32, kernel_size=(5,5)))
generator.add_module(next(ix), nn.ELU())
generator.add_module(next(ix), nn.ConvTranspose2d(32, 32, kernel_size=(5,5)))
generator.add_module(next(ix), nn.ELU())

generator.add_module(next(ix), nn.Conv2d(32, 3, kernel_size=(5,5)))

if use_cuda: generator.cuda()

In [ ]:
generated_data = generator(sample_noise_batch(5))
assert tuple(generated_data.shape)[1:] == IMG_SHAPE, "generator must output an image of shape %s, but instead it produces %s"%(IMG_SHAPE,generated_data.shape)

### Дискриминатор
* Дискриминатор это обычная свёрточная сеть, содержащая свёрточные (convolution) слои и слои субдискретизации (pooling)
* Сеть не включает в себя dropout/batchnorm слои чтобы избежать сложностей при обучении.
* Мы дополнительно осуществляем регуляризацию предвыходного слоя чтобы предотвратить чрезмерную уверенность дискриминатора при предсказании.

In [ ]:
def sample_data_batch(batch_size):
    idxs = np.random.choice(np.arange(data.shape[0]), size=batch_size)
    batch = Variable(torch.FloatTensor(data[idxs]))
    return batch.cuda() if use_cuda else batch.cpu()

# a special module that converts [batch, channel, w, h] to [batch, units]
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
discriminator = nn.Sequential()

### YOUR CODE - create convolutional architecture for discriminator
### Note: please start simple. A few convolutions & poolings would do, inception/resnet is an overkill

discriminator.add_module("disc_logit", nn.Linear(<???>, 1))

if use_cuda: discriminator.cuda()

In [ ]:
discriminator(sample_data_batch(5))

# Обучение

Мы будем одновременно обучать две сети:
* Обучаем __дискриминатор__ чтобы он мог лучше отличать истинные примеры от __текущих__ сгенерированных
* Обучаем __генератор__ чтобы обмануть дискриминатор, заставив думать что сгенерированные примеры являются истинными
* Поскольку дискриминатор является дифференцируемой нейронной сетью, мы будем обучать обе сети при помощи градиентного спуска.

![img](https://s24.postimg.org/cw4nognxx/gan.png)

Обучение выполняется итеративно до тех пор, пока дискриминатор будет не в состоянии найти разницу между искусственными и истинными примерами, или пока у вас не закончится терпение.


### Советы:
* Регуляризуйте выходные веса выходного слоя дискриминатора чтобы предотвратить перекос значений
* Обучайте генератор используя __adam__ чтобы ускорить процесс обучения. Обучайте дискриминатор при помощи стохастического градиентного спуска чтобы предотвратить проблемы с моментом.
* Больше советов: https://github.com/soumith/ganhacks


In [ ]:
def generator_loss(noise):
    """
    1. generate data given noise
    2. compute log P(real | gen noise)
    3. return generator loss (should be scalar)
    """
    generated_data = <generate data given noise>
    
    disc_on_generated_data = <discriminator's opinion on generated data>
    
    logp_gen_is_real = F.logsigmoid(disc_on_generated_data)
    
    loss = - <generator loss>
    
    return loss

In [ ]:
loss = generator_loss(sample_noise_batch(32))

print(loss)

assert len(loss.shape) == 1 and loss.shape[0] == 1, "loss must be scalar"

In [ ]:
def discriminator_loss(real_data, generated_data):
    """
    1. compute discriminator's output on real & generated data
    2. compute log-probabilities of real data being real, generated data being fake
    3. return discriminator loss (scalar)
    """
    disc_on_real_data = <discriminator's opinion on real data>
    disc_on_fake_data = <discriminator's opinion on generated data>
    
    logp_real_is_real = F.logsigmoid(disc_on_real_data)
    logp_gen_is_fake = F.logsigmoid(- logp_gen_is_fake)
    
    loss = <discriminator loss>
    return loss

In [ ]:
loss = discriminator_loss(sample_data_batch(32), 
                   generator(sample_noise_batch(32)))

print(loss)

assert len(loss.shape) == 1 and loss.shape[0] == 1, "loss must be scalar"

### Дополнительные функции
Здесь мы определим ряд функций-помощников, которые позволят получать текущие распределения данных и батчи для обучения.

In [ ]:
def sample_images(nrow,ncol, sharp=False):
    images = generator(sample_noise_batch(batch_size=nrow*ncol))
    images = images.data.cpu().numpy().transpose([0,2,3,1])
    if np.var(images)!=0:
        images = images.clip(np.min(data),np.max(data))
    for i in range(nrow*ncol):
        plt.subplot(nrow,ncol,i+1)
        if sharp:
            plt.imshow(images[i],cmap="gray", interpolation="none")
        else:
            plt.imshow(images[i],cmap="gray")
    plt.show()

def sample_probas(batch_size):
    plt.title('Generated vs real data')
    D_real = F.sigmoid(discriminator(sample_data_batch(batch_size)))
    generated_data_batch = generator(sample_noise_batch(batch_size))
    D_fake = F.sigmoid(discriminator(generated_data_batch))
    
    plt.hist(D_real.data.cpu().numpy(),
             label='D(x)', alpha=0.5,range=[0,1])
    plt.hist(D_fake.data.cpu().numpy(),
             label='D(G(z))',alpha=0.5,range=[0,1])
    plt.legend(loc='best')
    plt.show()

### Обучение
Основной цикл.
Мы циклически обучаем генератор и дискриминатор, и просматриваем результаты каждые N итераций.

In [ ]:
#optimizers
disc_opt = torch.optim.SGD(discriminator.parameters(), lr=5e-3)
gen_opt = torch.optim.Adam(generator.parameters(), lr=1e-4)

In [ ]:
from IPython import display
from tqdm import tnrange
batch_size = 100

for epoch in tnrange(50000):
    
    # Train discriminator
    for i in range(5):
        real_data = sample_data_batch(batch_size)
        fake_data = generator(sample_noise_batch(batch_size))
        loss = discriminator_loss(real_data, fake_data)
        disc_opt.zero_grad()
        loss.backward()
        disc_opt.step()
        
    # Train generator
    noise = sample_noise_batch(batch_size)
    loss = generator_loss(noise)
    gen_opt.zero_grad()
    loss.backward()
    gen_opt.step()
    
    if epoch %100==0:
        display.clear_output(wait=True)
        sample_images(2,3,True)
        sample_probas(1000)
        

In [ ]:
#The network was trained for about 15k iterations. 
#Training for longer yields MUCH better results
plt.figure(figsize=[16,24])
sample_images(16,8)